# Exercise 2 - Working with Excel
In dieser Übung wird ein Excel als Input für die Beschreibung eines Gebäudes verwendet




### Excel einlesen
Lesen Sie aus dem Excel **data/building.xlsx** das Sheet "thermal_hull" in einen Pandas Dataframe namens *hull* ein 
> Hint: https://pythonbasics.org/read-excel/

In [ ]:
!pip install openpyxl
#or
!conda install openpyxl

In [ ]:
import pandas as pd

hull = pd.read_excel("data/building.xlsx", sheet_name="thermal_hull")
hull

Ermitteln Sie die insgesamte Fläche der thermischen Hülle $A_B$ 

In [ ]:
A_B = hull["Fläche"].sum()
print(f"A_B = {A_B:.2f} m²")

Erstellen Sie eine Funktion *thermal_hull*, die als Input den Pfad des building.xlsx entgegennimmt und einen pandas Dataframe ausgibt. Testen Sie die Funktion mit der Datei  **data/building2.xslx**

In [ ]:
def thermal_hull(path):
    return pd.read_excel(path, sheet_name="thermal_hull")

In [ ]:
hull = thermal_hull("data/building2.xlsx")
hull

### Transmissions-Leitwert der Gebäudehülle
Ermitteln Sie den Transmission-Leitwert der Bauteile
$L_B =\sum_if_iU_iA_i[W/K]$


In [ ]:
hull["L_B"] = hull["Fläche"] * hull["U-Wert"] * hull["Temperatur-Korrekturfaktor"]
hull

In [ ]:
L_B = hull.L_B.sum()
print(f"L_B = {L_B:.2f} W/K")

Ermitteln Sie die Leitwert-Zuschläge $L_\psi+L_\chi$ entsprechend dem vereinfachten Ansatz:
$L_\psi+L_\chi = 0.2(0.75 - \frac{L_B}{A_B})A_B \geq 0  [W/K]$

In [ ]:
L_PX = max(0, (0.2*(0.75-L_B/A_B)*L_B))

print(f"L_PX = {L_PX:.2f} W/K")

Erstellen Sie eine Funktion *L_T*, die einen *thermal_hull* Pandas Dataframe entgegennimmt und den Transmissions-Leitwert der Gebäudehülle $L_T = L_B + L_\psi+L_\chi$ zurückgibt. Testen Sie die Funktion mit dem davor erstellten Dataframe

In [ ]:
def L_T(hull_df): # expects a pandas dataframe as input
    A_B = hull_df["Fläche"].sum()
    hull_df["L_B"] = hull_df["Fläche"] * hull_df["U-Wert"] * hull_df["Temperatur-Korrekturfaktor"]
    L_B = hull_df.L_B.sum()
    L_PX = max(0, (0.2*(0.75-L_B/A_B)*L_B))
    L_T = L_B + L_PX
    return L_T

In [ ]:
print(f"L_T = {L_T(hull):.2f} W/K")

Verwenden Sie abschließend die beiden erstellten Funktionen *thermal_hull* und *L_T* um in je einer Zeile die Leitwert der beiden Test-Gebäude *building.xlsx* und *building2.xlsx* darzustellen 

In [ ]:
print("building.xlsx:",L_T(thermal_hull("data/building.xlsx")))
print("building2.xlsx:",L_T(thermal_hull("data/building2.xlsx")))

### Fenster
Ergänzen Sie im *hull* Dataframe zwei Zeilen: "AW (Opak)" und "Fenster". Teilen Sie dazu die Brutto-Außenwand im Verhältnis 60:40 auf und ergänzen Sie für die Fenster einen U-Wert von 1.5 W/m²K 

In [ ]:
hull = thermal_hull("data/building.xlsx")
aw_A = hull.loc[0,"Fläche"]
aw_A


In [ ]:
aw_opak_A = aw_A * 0.6
fenster_A = aw_A * 0.4
fenster_A

In [ ]:
hull

In [ ]:
L_T(hull)

In [ ]:
aw_opak = dict(zip(hull.columns,["AW (opak)", aw_opak_A, hull.loc[0,"U-Wert"], hull.loc[0,"Temperatur-Korrekturfaktor"]]))
aw_opak

In [ ]:
hull = hull.append(aw_opak, ignore_index = True)
hull

In [ ]:
fenster = dict(zip(hull.columns,["Fenster", fenster_A, 1.5, hull.loc[0,"Temperatur-Korrekturfaktor"]]))
hull = hull.append(fenster, ignore_index = True)
hull

In [ ]:
L_T(hull)

Erstellen Sie eine Funktion *insert_windows*, die diese Ergänzungen im Dataframe durchführen und  den U-Wert des fensters und den Fensterflächen-Anteil als Variable nehmen und einen aktualisierten Dataframe zurückgeben. Löschen Sie davor den Eintrag AW (brutto). Testen Sie, ob die Leitwertberechnung L_T() mit dem aktualisierten Dataframe funktioniert.  
> Hint: https://www.shanelynn.ie/pandas-drop-delete-dataframe-rows-columns/


In [ ]:
def insert_windows(hull_df, u_f, ff_anteil):
    aw_A = hull_df.loc[0,"Fläche"]
    aw_opak_A = aw_A * (1-ff_anteil)
    fenster_A = aw_A * ff_anteil
    
    aw_opak = dict(zip(hull_df.columns,["AW (opak)", aw_opak_A, hull_df.loc[0,"U-Wert"], hull_df.loc[0,"Temperatur-Korrekturfaktor"]]))
    fenster = dict(zip(hull_df.columns,["Fenster", fenster_A, u_f, hull_df.loc[0,"Temperatur-Korrekturfaktor"]]))
    
    hull_df = hull_df.append(aw_opak, ignore_index = True)
    hull_df = hull_df.append(fenster, ignore_index = True)
    hull_df.drop(hull_df.index[0], inplace=True)
    return hull_df



In [ ]:
hull2 = insert_windows(thermal_hull("data/building.xlsx"), u_f=1.5, ff_anteil=0.4)
hull2

In [ ]:
L_T(hull2)